# Оптимизация стоимости виртуальных машин при миграции в облако с использованием or-tools

## Сценарий

В [прошлый раз](/vmOptimization-basics.ipynb) мы оптимизировали общую стоимость при помощи minizinc. Это требовало установки дополнительного программного обеспечения, что, не очень удобно и, может быть, не совсем понятно. В этот раз мы хотим получить тот же результат, но "родными" средствами .NET. Для этого будем использовать [or-tools](https://developers.google.com/optimization/introduction/overview) - SMT солвер от Google.

> OR-Tools is open source software for combinatorial optimization, which seeks to find the best solution to a problem out of a very large set of possible solutions. Here are some examples of problems that OR-Tools solves:
>
> - Vehicle routing: Find optimal routes for vehicle fleets that pick up and deliver packages given constraints (e.g., "this truck can't hold more than 20,000 pounds" or "all deliveries must be made within a two-hour window").
> - Scheduling: Find the optimal schedule for a complex set of tasks, some of which need to be performed before others, on a fixed set of machines, or other resources.
> - Bin packing: Pack as many objects of various sizes as possible into a fixed number of bins with maximum capacities.

В нашем случае смысл в том, что or-tools solver имеет .NET интерфейс, коотрый позволяет нам использовать его в PowerShell почти без дополнительных усилий.

> Один неприятный момент все же есть. Сам солвер написан на С. Интерфейс на .NET - всего лишь обертка. Поэтому, чтобы сорвер работал, нам нужен [vc runtime](https://docs.microsoft.com/en-us/cpp/windows/latest-supported-vc-redist?view=msvc-170). Однако, для упрощения задачи я включаю необходимые библиотеки в состав модуля

Для упрощения, я сделал небольшой PowerShell модуль - [AzureVmCalc](https://github.com/eosfor/AzureVmCalc). Он содержит в себе все, что нужно. Установим его

> На данный момент модуль доступен в alpha версии. Процесс еще идет

In [ ]:
Install-Module AzureVmCalc -AllowPrerelease -Scope CurrentUser

Импортируем его

In [ ]:
Import-Module AzureVmCalc

Подготовим данные.

In [ ]:
$sourceVMs = Import-Csv ".\vm-optimization-ortools\vmdata.csv"
$sourceVMs  | % { $_.cpu = [int]$_.cpu; $_.ram = [int]$_.ram; $_.datadisk = [int]$_.datadisk; }

$targetSizes = import-csv ".\vm-optimization-ortools\vmCostACUData.csv"

Файл `vmdata.csv` должен содержать информацию о размерах исходных виртуальных или физических машин. Файл `vmCostACUData` содержит необходимые данные о стоимости всех типов виртуальных машин из определенного региона Azure.

Теперь можно запустить соотвествующую команду

In [ ]:
$x = Start-OrToolsModelCalculation -SourceVM $sourceVMs -TargetVM $targetSizes

Команда принимает исходный список, и возможные размеры и возвращает массив с результатами. На самом деле моделирование происходит в два этапа. Как и раньше, сначала модель пытается  максимизировать суммарную производительность всех машин. Затем, фиксирует это значение и пытается минимизировать суммарную стоимость, не снижая значения производительности. В итоге результат содержит результаты рассчета первого и второго этапа, для сравнения.

In [ ]:
$x


VmMappingResult
---------------
{Standard_E96-48as_v4, Standard_E96-48as_v4, Standard_E96-48as_v4, Standard_E96-48as_v4…}
{Standard_E4-2as_v4, Standard_E4-2as_v4, Standard_E4-2as_v4, Standard_E8-4as_v4…}



In [ ]:
$view = $x[1].VmMappingResult | select vmid, cpu, ram, name, vCPUs, MemoryGB, vCPUsPerCore, ACUs, retailPrice | ConvertTo-Html -Fragment
[Microsoft.DotNet.Interactive.Kernel]::HTML($view) | Out-Display

vmid,cpu,ram,Name,vCPUs,MemoryGB,vCPUsPerCore,ACUs,retailPrice
vmN1,2,12,Standard_E4-2as_v4,4,32,2,230,0.436
vmN2,2,24,Standard_E4-2as_v4,4,32,2,230,0.436
vmN3,2,4,Standard_E4-2as_v4,4,32,2,230,0.436
vmN4,6,23,Standard_E8-4as_v4,8,64,2,230,0.872
vmN5,3,16,Standard_E4-2as_v4,4,32,2,230,0.436
vmN6,2,32,Standard_E4-2as_v4,4,32,2,230,0.436
vmN7,2,7,Standard_E4-2as_v4,4,32,2,230,0.436
vmN8,3,32,Standard_E4-2as_v4,4,32,2,230,0.436
vmN9,2,8,Standard_E4-2as_v4,4,32,2,230,0.436
vmN10,1,40,Standard_E8-4as_v4,8,64,2,230,0.872
